<a href="https://colab.research.google.com/github/MyNameIsEldar/Movie_Recommender/blob/main/rec_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🎥Movie Recommender🎥
Recommendation System using deep learning

##✔️Task

Required to build embeddings for each user and movie using **neural networks methods**. These embeddings will be used to find more relevant film for user.

For this task I will use **full MovieLens dataset**.

##📑Imports

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error